## 4. Predicción de fugas de clientes en una compañía telefónica (7 puntos)

Una compañía telefónica está interesada en que desarrollemos un modelo que prediga los **100 clientes actuales** (dataset de explotaación) que tienen más probabilidad de abandonar la compañía. Para ello nos proporcionan una base de datos **fuga_clientes_empresa_telefonica_construccion.csv** con casos etiquetados, que usaremos para construir nuestro modelo de predicción.

Los campos de esta base de datos son:

* **Customer ID**

* **network_age**: antigüedad del cliente en días

* **Customer tenure in months:** antigüedad del cliente en meses

* **Total Spend in Months 1 and 2:** gasto total del cliente en los meses de referencia 1 y 2

* **Total SMS Spend:** gasto total en SMS

* **Total Data Spend:** gasto total en datos/internet

* **Total Data Consumption:** consumo total de datos (en KB) durante el período de estudio

* **Total Unique Calls:** número total de llamadas únicas

* **Total Onnet spend:** gasto total en llamadas a otros usuarios de la misma red de telefonía

* **Total Offnet spend:** gasto total en llamadas a otros usuarios de redes de telefonía diferentes

* **Total Call centre complaint calls:** número de llamadas de queja al call center

* **Network type subscription in Month 1:** suscripción de tipo de red en el mes 1. Esto indica la suscripción de red preferida de un cliente, lo que puede indicar su tipo de dispositivo: servicio 2G o 3G

* **Network type subscription in Month 2:** igual que el anterior pero en el mes posterior

* **Churn Status**: el valor es 1 si el cliente abandona la compañía telefónica, 0 si permanece en ella

* **Most Loved Competitor network in Month 1:** qué otro proveedor de la competencia prefiere el cliente en el mes 1. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

* **Most Loved Competitor network in Month 2:** qué otro proveedor de la competencia prefiere el cliente en el mes 2. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

La variable a predecir es **Churn Status**: el valor es 1 si el cliente **abandona** la compañía, 0 si no la abandona.

La compañía también nos proporciona otra base de datos, **fuga_clientes_empresa_telefonica_explotacion.csv**, con información sobre clientes de los que no se sabe ahora mismo si van a permanecer o no en la compañía. Por tanto en esta segunda base de datos todos los valores de la columna **Churn Status** son missing values (NaN).

La compañía nos pide que proporcionemos los IDs de los 100 clientes de la base de datos de explotación que con mayor probabilidad vayan a abandonar la compañía. Para ello proporcionaremos como entregable a la compañía un archivo csv con una sola columna, **Customer ID**, y 100 filas que contengan los IDs de los clientes seleccionados.

El fichero **ejemplo_fichero_predicciones.csv** contiene un ejemplo con el formato solicitado para este archivo.

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [ ]:
datos_construccion = pd.read_csv("./fuga_clientes_empresa_telefonica_construccion.csv")
datos_explotacion  = pd.read_csv("./fuga_clientes_empresa_telefonica_explotacion.csv")

In [ ]:
datos_construccion.columns

In [ ]:
datos_explotacion.columns

In [ ]:
# chequeo de que las columnas son exactamente las mismas en los dos ficheros:

sum(datos_construccion.columns != datos_explotacion.columns)

In [ ]:
# la columna a predecir es "Churn Status"
# si es 1, el cliente se va de la compañía
# si es 0, el cliente se queda

# Esta columna se sabe en el dataset de construcción (ejemplos de clientes pasados):
datos_construccion["Churn Status"].values[:10]

In [ ]:
# Sin embargo no se sabe en el dataset de explotación (clientes actuales):
datos_explotacion["Churn Status"].values[:10]

In [ ]:
nombres_atrs = list(datos_construccion.columns)

# We have removed Churn Status because it is what we want to predict, so it cannot be in the input data
nombres_atrs.remove("Churn Status")

# We have removed Customer ID because it does not affect that a client leaves or stays in the company
nombres_atrs.remove("Customer ID")
print(nombres_atrs)

X = datos_construccion[nombres_atrs].values
# Churn Status is the value we want to predict
y = datos_construccion["Churn Status"].values

In [ ]:
# Testing the efficiency of the neural network
clf_nn = MLPClassifier(hidden_layer_sizes=(60),
                       max_iter=1000,
                       alpha=0)

scores = cross_val_score(clf_nn,
                         X,
                         y,
                         cv=5,
                         )


print("Score global del modelo: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

In [ ]:
# Testing the efficiency of the k-nn algorithm
clf_knn = KNeighborsClassifier(n_neighbors=27)

scores = cross_val_score(clf_knn,
                         X,
                         y,
                         cv=5,
                         )


print("Score global del modelo: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

In [ ]:
# Testing the efficiency of the decision tree
clf_dt = DecisionTreeClassifier(criterion='entropy', max_depth=7)

scores = cross_val_score(clf_dt,
                         X,
                         y,
                         cv=5,
                         )


print("Score global del modelo: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

In [ ]:
# Testing the efficiency of gaussian naive bayes
clf_gnb = GaussianNB()

scores = cross_val_score(clf_gnb,
                         X,
                         y,
                         cv=5,
                         )


print("Score global del modelo: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

In [ ]:
# Testing the efficiency of logistic regression
clf_lr = LogisticRegression(C=1e10,solver='lbfgs')

scores = cross_val_score(clf_lr,
                         X,
                         y,
                         cv=5,
                         )


print("Score global del modelo: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

After testing which prediction algorithm was the best for this problem, we reached the conclusion that a decision tree with depth 7 was the best option. All the values in the code were adapted to get the best result possible with each one of them.

Once we decided that the best option was to use a decision tree with depth 7, we fit it to the data so it "remembers" how to guess which customer was going to leave the company. After that, we load the new dataset (removing custormer id and churn status) and we use the desicion tree to predict the clients who will leave the company. This will give us an array of churn statuses (0s and 1s). With this array we can use the customer ids from fuga_clientes_empresa_telefonica_explotacion.csv and create the prediction.csv file where we write the ids of the clients that may leave the company as in ejemplo_fichero_predicciones.csv.

In [ ]:
# We remove Churn Status and Customer ID from the input data as we did previously
nombres_prediction = list(datos_explotacion.columns)
nombres_prediction.remove("Churn Status")
nombres_prediction.remove("Customer ID")

clf_dt.fit(X, y)

predictX = datos_explotacion[nombres_prediction].values
predictY = clf_dt.predict(predictX)

print(predictY)

f = open("prediction.csv", "w")
f.write("Customer ID\n")

# Write to the csv file the customers that may leave
for i in range(len(datos_explotacion["Customer ID"])):
    if predictY[i] == 1:
        f.write(str(datos_explotacion["Customer ID"].values[i]) + "\n")

f.close()